In [ ]:

!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
# Standard imports
import os
from groq import Groq

# Setup Groq API key for test/demo
os.environ['GROQ_API_KEY'] = "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"

# Initialize Groq client
client = Groq(api_key=os.environ['GROQ_API_KEY'])
MODEL = "llama-3.3-70b-versatile"


In [ ]:
# Helper functions for truncation
def truncate_by_turns(history, n):
    return history[-n:]

def truncate_by_chars(history, max_chars):
    result = []
    count = 0
    for msg in reversed(history):
        msg_len = len(msg['content'])
        if count + msg_len > max_chars:
            break
        result.insert(0, msg)
        count += msg_len
    return result

def truncate_by_words(history, max_words):
    result, count = [], 0
    for msg in reversed(history):
        words = len(msg['content'].split())
        if count + words > max_words:
            break
        result.insert(0, msg)
        count += words
    return result

def summarize(history):
    system_prompt = {"role": "system", "content": "Summarize the conversation so far in 2 concise sentences."}
    summary_msgs = [system_prompt] + history
    resp = client.chat.completions.create(
        model=MODEL,
        messages=summary_msgs
    )
    return resp.choices[0].message.content


In [ ]:
# A conversation runner with truncation and periodic summarization
def conversation_demo(history, user_msg, k=3, trunc_type="turns", limit=5, summarization_every=3):
    # Add user message and assistant response
    history.append({"role": "user", "content": user_msg})
    assistant = client.chat.completions.create(model=MODEL, messages=history)
    ai_msg = assistant.choices[0].message.content
    history.append({"role": "assistant", "content": ai_msg})

    # Truncate history
    if trunc_type == "turns":
        history = truncate_by_turns(history, limit)
    elif trunc_type == "chars":
        history = truncate_by_chars(history, limit)
    elif trunc_type == "words":
        history = truncate_by_words(history, limit)

    # Summarize periodically
    if len(history) // 2 % summarization_every == 0:
        summary = summarize(history)
        history = [{"role": "system", "content": f"Summary: {summary}"}]

    return history

# --- Demo Run ---
# Initial empty conversation
history = [{"role": "system", "content": "You are a friendly AI assistant."}]
sample_user_msgs = [
    "Hi there, can you tell me a fun fact?",
    "What's the weather like in Paris?",
    "Can you summarize our conversation?",
    "Who won the last football world cup?",
    "What is the capital of Japan?"
]
# Demo, show conversation at each step with truncation and every-3rd-run summarization
for i, msg in enumerate(sample_user_msgs):
    history = conversation_demo(history, msg, k=3, trunc_type="turns", limit=4, summarization_every=3)
    print(f"\n--- After user message {i + 1} ---\n")
    for h in history:
        print(f"{h['role'].capitalize()}: {h['content']}")



--- After user message 1 ---

System: You are a friendly AI assistant.
User: Hi there, can you tell me a fun fact?
Assistant: Here's a fun fact: Did you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal! Isn't that cool?

--- After user message 2 ---

User: Hi there, can you tell me a fun fact?
Assistant: Here's a fun fact: Did you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal! Isn't that coo

**Task 2: JSON Schema Classification & Information Extraction**

In [ ]:
# Demo sample chats
sample_chats = [
    "Hello, I'm Priya Shah from Mumbai. My email is priya@gmail.com and I'm 29 years old. Contact me on 9123456789.",
    "My name is John, 34 years old, living in London. You can reach me at john_doe22@icloud.com or call 44011223344.",
    "This is Maria, based in São Paulo, age 41. My number is +5511998877665, and my mail is maria.silva@email.com."
]


In [ ]:
# Define JSON schema dictionary (no Function import)
info_schema = {
    "name": "extract_user_info",
    "description": "Extract user contact and demographic info from chat.",
    "parameters": {
        "type": "object",
        "properties": {
            "name":       {"type": "string", "description": "User's name"},
            "email":      {"type": "string", "description": "User's email"},
            "phone":      {"type": "string", "description": "User's phone"},
            "location":   {"type": "string", "description": "User's location"},
            "age":        {"type": "integer", "description": "User's age"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

# Extraction and validation loop
import json

for chat in sample_chats:
    chat_msgs = [
        {"role": "system", "content": "Extract user's name, email, phone, location, and age from their message."},
        {"role": "user", "content": chat}
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=chat_msgs,
        functions=[info_schema],
        function_call={"name": "extract_user_info"}
    )

    # Extract and validate output
    function_args = response.choices[0].message.function_call.arguments
    try:
        parsed = json.loads(function_args)
        valid = all(parsed.get(field) for field in ["name", "email", "phone", "location", "age"])
        print(f"Chat: {chat}\nExtracted: {parsed}\nValid: {valid}\n")
    except Exception as e:
        print(f"Failed to parse/validate: {e}")


Chat: Hello, I'm Priya Shah from Mumbai. My email is priya@gmail.com and I'm 29 years old. Contact me on 9123456789.
Extracted: {'age': 29, 'email': 'priya@gmail.com', 'location': 'Mumbai', 'name': 'Priya Shah', 'phone': '9123456789'}
Valid: True

Chat: My name is John, 34 years old, living in London. You can reach me at john_doe22@icloud.com or call 44011223344.
Extracted: {'age': 34, 'email': 'john_doe22@icloud.com', 'location': 'London', 'name': 'John', 'phone': '44011223344'}
Valid: True

Chat: This is Maria, based in São Paulo, age 41. My number is +5511998877665, and my mail is maria.silva@email.com.
Extracted: {'age': 41, 'email': 'maria.silva@email.com', 'location': 'São Paulo', 'name': 'Maria', 'phone': '+5511998877665'}
Valid: True



In [17]:
# Yarstick Groq Assignment
# Conversation Management & Classification using Groq API
# This file is a self-contained Google Colab-friendly Python script / notebook

# -----------------------------
# README.md (for GitHub)
# -----------------------------

readme_content = """# Yarstick Groq API Assignment

This repository contains my submission for the **Conversation Management & Classification** assignment using the **Groq API** with OpenAI-compatible SDK.

## 📌 Tasks
- **Task 1**: Manage conversation history, truncation, and periodic summarization.
- **Task 2**: JSON schema classification & information extraction with function-calling.

## 🚀 How to Run
1. Open the notebook in **Google Colab** or Jupyter Notebook.
2. Install requirements:
   ```bash
   pip install -r requirements.txt
   ```
3. Set your **Groq API key**:
   ```python
   import os
   os.environ["GROQ_API_KEY"] = "<your-api-key>"
   ```
4. Run all cells to reproduce results.

## 📂 Repository Structure
```
yarstick-groq-assignment/
│── yarstick_groq_assignment.ipynb   # Main notebook
│── requirements.txt                 # Python dependencies
│── README.md                        # This file
```

## 🔑 Notes
- API key is **not included** in this repo. Set it manually in Colab or your local environment.
- The notebook demonstrates:
  - Conversation management with summarization every k turns.
  - Truncation by turns and by character length.
  - JSON schema-based structured extraction of user details.
- Outputs are shown in the notebook for evaluation.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

# -----------------------------
# requirements.txt (for GitHub)
# -----------------------------

requirements_content = """openai
requests
jsonschema
"""

with open("requirements.txt", "w") as f:
    f.write(requirements_content)

print("README.md and requirements.txt have been generated. Upload them to GitHub along with your notebook.")


README.md and requirements.txt have been generated. Upload them to GitHub along with your notebook.
